In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
clients, test_x, test_y = utils.set_data(True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [4]:
selected_model = Supported_modles.logistic_regression

In [5]:
f1_before = []
for client in clients:
    # client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    # client.prep_data()
    client.init_empty_model(selected_model,0.01)
    client.train_model()
    y_hat = client.model.predict(test_x)
    f1_before.append(f1_score(test_y,y_hat, average="binary"))

In [32]:
f1_before

[0.9785996409335727,
 0.9922530664945125,
 0.9863250822226068,
 0.9839999999999999,
 0.9992234015014237]

In [33]:
selected_model = Supported_modles.SGD_classifier

In [34]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,0.1)

In [35]:
fedavg = Fedavg("global", 0.1)
fedavg.init_global_model(selected_model, None,78)

[Errno 98] Address already in use
Waitiing for a Connection..


In [36]:
number_of_rounds = 20
batch_size = 0.1
epochs = 30
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(_, end=' ')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # print(round_weights)
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    if _ % 10 == 0:
        f1_fedavg = []
        for client in clients:
            x = test_x
            y_hat = fedavg.model.predict(x)
            f1_fedavg.append(f1_score(test_y,y_hat))
        print(f1_fedavg)

0 [0.9392613023144262, 0.9392613023144262, 0.9392613023144262, 0.9392613023144262, 0.9392613023144262]
1 2 3 4 5 6 7 8 9 10 [0.9396607322304381, 0.9396607322304381, 0.9396607322304381, 0.9396607322304381, 0.9396607322304381]
11 12 13 14 15 16 17 18 19 

In [26]:
f1_fedavg = []
for client in clients:
    x = test_x
    y_hat = fedavg.model.predict(x)
    f1_fedavg.append(f1_score(test_y,y_hat))
print(f1_fedavg)

[0.9401920728489801, 0.9401920728489801, 0.9401920728489801, 0.9401920728489801, 0.9401920728489801]


In [24]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference -0.032333033322146876
Difference -0.030811689037832046
Difference -0.030906939406662115
Difference -0.03257258734514046
Difference -0.03213733712433087
